In [1]:
from src.models.our_model import OurModel
from src.models.pretrained_models import VGG16Pretrained, ResNetPretrained
import torch
from src.models.ensemble import HardVotingEnsemble, SoftVotingEnsemble
from src.utils import load_data, evaluate_model
from src.model_trainer import ModelTrainer
from src.transformations import normalized_simple_transform
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)
torch.manual_seed(123)
print(device)
torch.set_num_threads(14)

cpu


In [ ]:
test_loader = load_data('./data/test', batch_size=256, shuffle=True, transform=normalized_simple_transform(), num_workers=1)

In [3]:
vgg16 = VGG16Pretrained(device="cpu")  
vgg16.load_state_dict(torch.load("./saved_models/vgg_pretrained/regularization_params_001_2/VGG16Pretrained_10.pth", map_location=device))
vgg16.to(device)

c:\Users\karim\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\karim\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG16Pretrained(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
    

In [4]:
ourmodel = OurModel(aux_enabled=False, se_squeeze=8)  
ourmodel.load_state_dict(torch.load("./saved_models/ourmodel/combined_20/OurModel_16.pth", map_location=device))
ourmodel.to(device)

OurModel(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (inception1): Inception(
    (branch1): Sequential(
      (0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch2): Sequential(
      (0): Conv2d(64, 12, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (branch3): Sequential(
      (0): Conv2d(64, 12, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(12, ep

In [5]:
base_models = [ourmodel, vgg16]

In [6]:
hard_vot_ensemble = HardVotingEnsemble(base_models)
hard_vot_ensemble.to(device)

HardVotingEnsemble()

In [ ]:
metrics = evaluate_model(hard_vot_ensemble, test_loader, device)
print(metrics)

0 10 20 30 40 50 

In [ ]:
soft_vot_ensemble = SoftVotingEnsemble(base_models)
soft_vot_ensemble.to(device)

In [ ]:
metrics = evaluate_model(soft_vot_ensemble, test_loader, device)
print(metrics)